## THIS IS THE CODE FOR Ambition Jewels consider PO: NL PO# 101536 AMBITION (SHIMAYRA)-EDITING.pdf

### same code for retriving data from pdf as you sent obu:

In [1]:
pip install pandas regex pdfplumber openpyxl


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.




[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd  # #PDF TO TEXT PO1
import re
import pdfplumber

def extract_data_from_pdf(pdf_path):
    """
    Extract data from PDF file and return structured data
    """
    data = []
    
    try:
        with pdfplumber.open(pdf_path) as pdf:
            # Extract text from all pages
            full_text = ""
            for page in pdf.pages:
                full_text += page.extract_text() + "\n"
            
            print("Extracted text from PDF:")
            print("=" * 50)
            print(full_text)
            print("=" * 50)
            
            # Process the extracted text
            lines = full_text.split('\n')
            current_item = {}
            
            for line in lines:
                line = line.strip()
                if not line:
                    continue
                    
                # Extract SrNo (looking for lines starting with numbers)
                sr_no_match = re.match(r'^\s*(\d+)\s*', line)
                if sr_no_match:
                    if current_item and 'SrNo' in current_item:  # Save previous item if exists
                        data.append(current_item)
                    current_item = {'SrNo': sr_no_match.group(1), 'OrderQty': 2}
                    
                # Extract Article code (patterns like 9-DD106-YG-25, 9-DD106-YG-7)
                article_match = re.search(r'([A-Z0-9\-]+YG\-?\d*\.?\d+)', line)
                if article_match and 'ArticleCode' not in current_item:
                    current_item['ArticleCode'] = article_match.group(1)
                    
                # Extract StyleCode from Your reference (text after RSBR2074-01, RSBR2074-03, etc.)
                style_match = re.search(r'RSBR2074\-(\d+)\s+([A-Z0-9]+)', line)
                if style_match:
                    current_item['StyleCode'] = style_match.group(2)
                    
                # Extract ItemSize from description (looking for numbers like 0.25, 0.07)
                # Multiple patterns to catch different formats
                size_patterns = [
                    r'pendant\s+(\d+\.\d+)',  # pendant 0.25
                    r'(\d+\.\d+)\s*ct',       # 0.25 ct
                    r'YG[-\s]*(\d+\.\d+)',    # YG-0.25 or YG 0.25
                ]
                
                for pattern in size_patterns:
                    size_match = re.search(pattern, line, re.IGNORECASE)
                    if size_match and 'ItemSize' not in current_item:
                        current_item['ItemSize'] = size_match.group(1)
                        break
            
            # Add the last item
            if current_item and 'SrNo' in current_item:
                data.append(current_item)
                
    except Exception as e:
        print(f"Error reading PDF file: {e}")
        return []
    
    return data

def create_excel_dataframe(data):
    """
    Create DataFrame with specified columns
    """
    if not data:
        print("No data extracted from PDF")
        return pd.DataFrame()
    
    # Create DataFrame
    df = pd.DataFrame(data)
    
    # Reorder columns as requested
    columns_order = ['SrNo', 'StyleCode', 'ItemSize', 'OrderQty']
    
    # Only include columns that exist in the data
    final_columns = [col for col in columns_order if col in df.columns]
    
    # Add missing columns with empty values
    for col in columns_order:
        if col not in df.columns:
            df[col] = ""
    
    return df[columns_order]

# Main execution
if __name__ == "__main__":
    # Specify your PDF file path here
   
    #pdf_file_path = r'E:\OrderProcssing\data\damiani pc2\250915_ordini.pdf'  
    pdf_file_path = r'E:\OrderProcssing\data\damiani pc2\250917_ordini.pdf'
    #pdf_file_path = r'E:\OrderProcssing\data\damiani pc2\250925_ordini.pdf'
    # Change this to your actual PDF path
    
    # Extract data from PDF
    print(f"Reading PDF from: {pdf_file_path}")
    extracted_data = extract_data_from_pdf(pdf_file_path)
    
    if extracted_data:
        # Create DataFrame
        df = create_excel_dataframe(extracted_data)
        
        # Display the results
        print("\nExtracted Data:")
        print("=" * 50)
        print(df)
        
        # Save to Excel file
        output_file = 'extracted_data.xlsx'
        df.to_excel(output_file, index=False)
        print(f"\nData successfully saved to '{output_file}'")
        
        # Display basic statistics
        print(f"\nExtraction Summary:")
        print(f"Total records extracted: {len(df)}")
        print(f"Columns: {list(df.columns)}")
        
    else:
        print("No data was extracted from the PDF file.")
    
    # Display the DataFrame in notebook
    df

Reading PDF from: E:\OrderProcssing\data\damiani pc2\250917_ordini.pdf
Extracted text from PDF:
ORDINE DI ACQUISTO Standard (man)
Destinatario 40501295
Sede: 15048 VALENZA (AL) - P.za Damiano Grassi Damiani, 1 MESS.RS
P.IVA 01457570065 - R.E.A. 162836 AL - 3181 AL PUSHPAM COLLECTION BVBA
R.I.01457570065-Cap.soc. Euro 36.344.000 i.v.-M/AL012786
HOVENIERSSTRAAT 53,SUITE 907-909 9T
Fine Jewelry
B-2018 ANTWERP
- VALENZA 15048
P.za Damiano Grassi Damiani, 1
Tel.(0131) 92.96.11
Fax (0131) 94.16.53
damiani@damiani.it
Persona da contattare/Tel.
- MILANO 20121 VA: UA COLLEZ. SAL
Via Montenapoleone, 10
Federica Carnevale
Tel.(02) 46.71.61
Fax (02) 48.00.83.78 tel. 0131929600
Cd. dest. SDI: 7U7ZY9N
Indirizzo PEC: damiani@damiani.postecert.it
Numero Data Data consegna Linea
4400696111 17.09.2025 24.10.2025 Salvini gioielli
Mod. fornitore Ns. modello Descrizione Colore oro U.M. Quantità Importo
YR5444Z-8WG 20107689 anello ob brill. small mis.08 ORO BIANCO PZ 1 876,00
Mis. 08 Coll. MAGIA
1 USD 876,0

#logic for the AMBITION PO:

**PROMPT**:

consider the first cell in the @Ambition.ipynb file from which the text is extracted for the text extracted create a dataframe and arrange that text in the following columns in a dataframe and save that in a excel sheet, so add the cell accordingly at the end of the @Ambition.ipynb file to do so, the instructions for each columns to be filled are as follows,

COLUMNS:

for each item the details will start from the Sr.No. here the first item starts from "1." in the text and the data for that item lasts till  "**FOR SHIMAYRA**" is seen
1.SrNo: The first number that is present after the 5th line in the text for example here after the 5th line after the word "style" in next line "1." is the Sr.No. for this item similarly for the next ITem if at the end of the item after "**FOR SHIMAYRA**" is next Sr.No. if is not seen it then means that the file end there do not consider any thing after that.

2.StyleCode: the StyleCode for a item is the second last code in the Sr.No. line for example here the style code for first item is: 'ZNLSN00814' similarly find for other items

3.ItemSize:itemsize is the first decimal number that is present in the Sr.No. line for example here item size is "18.00"

4.OrderQty: the number that is present right after the ItemSize is the orderQty for example here order quantity is "38"

5.OrderItemPcs: leave this column blank

6.Metal: for different items there are difference Metal code for example for example if "SILV" is present in the text then it is AG925 so whenever "SILV" is encountered assign "AG925" as Metal for that item and similary "G14W" for Gold 14kt present in the text and "W" is the tone present which will depend on the tone column 

7.Tone: after the "SILV" word/letter is encountered for that item there will be the tone for example here "W" is the tone

8.ItemPoNo.: its the same for all the items here in the text at the top PO # : 101533 "101533" is the ItemPoNo

9.ItemRefNo: the first digits that are present just after the Sr.No. is the ItemRefNo for example here "101536/101" is the ItemRefNo for that item

10.StockType: leave it blank

11.MakeType: leave it blank

12.CustomerProductionInstruction: for example here "1/2 CT LCD BUTTERCUP ETERNIT NECK GSI CB" is the CustomerProductionInstruction for that item similarly find for other items too if present

13.SpecialRemarks: here "101536/101  LN2095SS-24 SILV  SZ-18inch,  DIA I SI2" is the SpecialRemarks for the item so find similarly where it is present in the text and map according to the items in the text.

14.DesignProductionInstruction: here "**GSI MIN CERT** **NO GSI# ENGRAVING ON PCS** **DIRECT SHIP TO CANADA**" is the DesignProductionInstruction for the item similarly find for other items too

15.StampInstruction: here "STAMP 925 LGD ANJ" is the StampInstruction similarly find for other items too

16.OrderGroup: leave it blank

17.Certificate: leave it blank

18.SKUNo: here "LN2095SS-24" is the SKUNo for the item similarly find for other items too

19.Basestoneminwt:leave it blank

20.Basestonemaxwt:leave it blank

21.Basemetalminwt:leave it blank

22.Basemetalmaxwt:leave it blank

23.Productiondeliverydate:leave it blank

24.Expecteddeliverydate:leave it blank

25. create a blank column:leave it blank

26.SetPrice:leave it blank

27.StoneQuality:leave it blank



In [6]:
# Data processing for PC2 - Damiani Purchase Order
import pandas as pd
import re

def process_pc2_damiani_data():
    """
    Process the extracted text from PC2 Damiani PDF and create DataFrame with specified columns
    """
    
    # The extracted text from the PDF (from the second cell output)
    extracted_text = """ORDINE DI ACQUISTO Standard (man)
Destinatario 40501295
Sede: 15048 VALENZA (AL) - P.za Damiano Grassi Damiani, 1 MESS.RS
P.IVA 01457570065 - R.E.A. 162836 AL - 3181 AL PUSHPAM COLLECTION BVBA
R.I.01457570065-Cap.soc. Euro 36.344.000 i.v.-M/AL012786
HOVENIERSSTRAAT 53,SUITE 907-909 9T
Fine Jewelry
B-2018 ANTWERP
- VALENZA 15048
P.za Damiano Grassi Damiani, 1
Tel.(0131) 92.96.11
Fax (0131) 94.16.53
damiani@damiani.it
Persona da contattare/Tel.
- MILANO 20121 VA: UA COLLEZ. SAL
Via Montenapoleone, 10
Federica Carnevale
Tel.(02) 46.71.61
Fax (02) 48.00.83.78 tel. 0131929600
Cd. dest. SDI: 7U7ZY9N
Indirizzo PEC: damiani@damiani.postecert.it
Numero Data Data consegna Linea
4400696111 17.09.2025 24.10.2025 Salvini gioielli
Mod. fornitore Ns. modello Descrizione Colore oro U.M. Quantità Importo
YR5444Z-8WG 20107689 anello ob brill. small mis.08 ORO BIANCO PZ 1 876,00
Mis. 08 Coll. MAGIA
1 USD 876,00
Annotazioni
Per la data di consegna indicata sull'ordine e con Voi concordata, si intende l'accettazione degli oggetti in Casa Damiani, una volta superato il
controllo qualità.
Condizioni di pagamento
Pagina
BONIFICO 120 GG DF
1
Il bonifico bancario verrà effettuato entro 15 giorni dalla scadenza di pagamento, in considerazione della gestione
amm.va e di tesoreria di
Termini di pagamento concordati in deroga, in considerazione della prassi commerciale, tempi di pagamento dei
clienti e natura del prodotto 1
*Il documento di trasporto e la fattura devono necessariamente riportare il numero completo del presente ordine onde evitare impossibilità di
registrazione e pagamento. Si richiede inoltre di mantenere la stessa sequenza riga per riga del presente ordine.
Si dichiara che Damiani s.p.a. svolge la propria attività in conformità alla legge, al Modello Organizzativo e al Codice Etico adottati ai sensi e per gli effetti del D.Lgs. 231/2001 ed esige che proprie
controparti contrattuali si conformino alla predetta normativa e al predetto Codice consultabile sul sito www.damiani.it specificando che l'eventuale comportamento contrario a detti principi potrà
comportare la risoluzione del contratto ai sensi dell'art. 1456 c.c. e l'adozione delle ulteriori misure previste dalla legge."""
    
    # Initialize the data list
    data = []
    
    # Find the ItemPoNo (Linea number)
    linea_match = re.search(r'(\d+)\s+\d{2}\.\d{2}\.\d{4}\s+\d{2}\.\d{2}\.\d{4}\s+Salvini gioielli', extracted_text)
    item_po_no = linea_match.group(1) if linea_match else ""
    
    # Find the item line after "U.M. Quantità Importo"
    item_line_match = re.search(r'U\.M\. Quantità Importo\n([^\n]+)', extracted_text)
    if item_line_match:
        item_line = item_line_match.group(1).strip()
        
        # Extract StyleCode (first two words)
        words = item_line.split()
        style_code = f"{words[0]} {words[1]}" if len(words) >= 2 else words[0] if words else ""
        
        # Extract ItemRefNo (third word - the number after StyleCode)
        item_ref_no = words[1] if len(words) >= 3 else ""
        
        # Extract ItemSize (number after "mis.")
        size_match = re.search(r'mis\.(\d+)', item_line)
        item_size = f"IT {size_match.group(1)}" if size_match else ""
        
        # Extract Tone (before "PZ")
        tone_match = re.search(r'(ORO\s+\w+)\s+PZ', item_line)
        tone = ""
        if tone_match:
            oro_type = tone_match.group(1)
            if "ORO BIANCO" in oro_type:
                tone = "WG"
            elif "ORO ROSE" in oro_type:
                tone = "PG"
            elif "ORO GIALLO" in oro_type:
                tone = "YG"
        
        # Extract OrderQty (number after "PZ")
        qty_match = re.search(r'PZ\s+(\d+)', item_line)
        order_qty = qty_match.group(1) if qty_match else ""
        
        # Create CustomerProductionInstruction based on tone
        customer_production_instruction = ""
        if tone == "WG":
            customer_production_instruction = "750 WHITE GOLD"
        elif tone == "PG":
            customer_production_instruction = "750 PINK GOLD"
        elif tone == "YG":
            customer_production_instruction = "750 YELLOW GOLD"
        
        # Create DesignProductionInstruction based on tone
        design_production_instruction = ""
        if tone == "WG":
            design_production_instruction = "WHITE RODIUM"
        elif tone == "PG":
            design_production_instruction = "NO RODIUM"
        elif tone == "YG":
            design_production_instruction = "NO RODIUM"
        
        # Create SpecialRemarks
        size_for_remarks = item_size.replace("IT ", "").replace(".", "")  # Remove IT prefix and decimal point
        special_remarks = f"SVP - {item_ref_no}, {customer_production_instruction}, {design_production_instruction}, ITALIAN SIZE - {size_for_remarks}"
        
        # Create the data dictionary with all required columns
        item_data = {
            'SrNo': 1,  # Incremental value starting from 1
            'StyleCode': style_code,
            'ItemSize': item_size,
            'OrderQty': order_qty,
            'OrderItemPcs': "",  # Leave blank
            'Metal': "G750",  # Fixed value
            'Tone': tone,
            'ItemPoNo.': item_po_no,
            'ItemRefNo': item_ref_no,
            'StockType': "",  # Leave blank
            'Priority': "REG",  # Fixed value
            'MakeType': "",  # Leave blank
            'CustomerProductionInstruction': customer_production_instruction,
            'SpecialRemarks': special_remarks,
            'DesignProductionInstruction': design_production_instruction,
            'StampInstruction': "750 SALVINI",  # Fixed value
            'OrderGroup': "",  # Leave blank
            'Certificate': "",  # Leave blank
            'SKUNo': "",  # Leave blank
            'Basestoneminwt': "",  # Leave blank
            'Basestonemaxwt': "",  # Leave blank
            'Basemetalminwt': "",  # Leave blank
            'Basemetalmaxwt': "",  # Leave blank
            'Productiondeliverydate': "",  # Leave blank
            'Expecteddeliverydate': "",  # Leave blank
            'SetPrice': "",  # Leave blank
            'StoneQuality': ""  # Leave blank
        }
        
        data.append(item_data)
        
        # Debug output to verify extraction
        print("Debug - Extracted values:")
        print(f"ItemPoNo: {item_po_no}")
        print(f"StyleCode: {style_code}")
        print(f"ItemRefNo: {item_ref_no}")
        print(f"ItemSize: {item_size}")
        print(f"Tone: {tone}")
        print(f"OrderQty: {order_qty}")
        print(f"CustomerProductionInstruction: {customer_production_instruction}")
        print(f"DesignProductionInstruction: {design_production_instruction}")
        print(f"SpecialRemarks: {special_remarks}")
        print("-" * 50)
    
    return data

# Process the data
processed_data = process_pc2_damiani_data()

# Create DataFrame
df = pd.DataFrame(processed_data)

# Display the DataFrame
print("Processed DataFrame:")
print("=" * 80)
print(df.to_string(index=False))
print("=" * 80)

# Save to Excel file
output_filename = 'PC2_Damiani_Processed_Data.xlsx'
df.to_excel(output_filename, index=False)

print(f"\nData successfully saved to '{output_filename}'")
print(f"Total items processed: {len(df)}")
print(f"Columns created: {len(df.columns)}")

# Display column names
print(f"\nColumn names:")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

df


Debug - Extracted values:
ItemPoNo: 4400696111
StyleCode: YR5444Z-8WG 20107689
ItemRefNo: 20107689
ItemSize: IT 08
Tone: WG
OrderQty: 1
CustomerProductionInstruction: 750 WHITE GOLD
DesignProductionInstruction: WHITE RODIUM
SpecialRemarks: SVP - 20107689, 750 WHITE GOLD, WHITE RODIUM, ITALIAN SIZE - 08
--------------------------------------------------
Processed DataFrame:
 SrNo            StyleCode ItemSize OrderQty OrderItemPcs Metal Tone  ItemPoNo. ItemRefNo StockType Priority MakeType CustomerProductionInstruction                                                  SpecialRemarks DesignProductionInstruction StampInstruction OrderGroup Certificate SKUNo Basestoneminwt Basestonemaxwt Basemetalminwt Basemetalmaxwt Productiondeliverydate Expecteddeliverydate SetPrice StoneQuality
    1 YR5444Z-8WG 20107689    IT 08        1               G750   WG 4400696111  20107689                REG                         750 WHITE GOLD SVP - 20107689, 750 WHITE GOLD, WHITE RODIUM, ITALIAN SIZE - 08 

,SrNo,StyleCode,ItemSize,OrderQty,OrderItemPcs,Metal,Tone,ItemPoNo.,ItemRefNo,StockType,...,Certificate,SKUNo,Basestoneminwt,Basestonemaxwt,Basemetalminwt,Basemetalmaxwt,Productiondeliverydate,Expecteddeliverydate,SetPrice,StoneQuality
0,1,YR5444Z-8WG 20107689,IT 08,1,,G750,WG,4400696111,20107689,,...,,,,,,,,,,


In [ ]:
#915 po
ORDINE DI ACQUISTO Standard (man)
Destinatario 40501295
Sede: 15048 VALENZA (AL) - P.za Damiano Grassi Damiani, 1 MESS.RS
P.IVA 01457570065 - R.E.A. 162836 AL - 3181 AL PUSHPAM COLLECTION BVBA
R.I.01457570065-Cap.soc. Euro 36.344.000 i.v.-M/AL012786
HOVENIERSSTRAAT 53,SUITE 907-909 9T
Fine Jewelry
B-2018 ANTWERP
- VALENZA 15048
P.za Damiano Grassi Damiani, 1
Tel.(0131) 92.96.11
Fax (0131) 94.16.53
damiani@damiani.it
Persona da contattare/Tel.
- MILANO 20121 VA: UA COLLEZ. SAL
Via Montenapoleone, 10
Federica Carnevale
Tel.(02) 46.71.61
Fax (02) 48.00.83.78 tel. 0131929600
Cd. dest. SDI: 7U7ZY9N
Indirizzo PEC: damiani@damiani.postecert.it
Numero Data Data consegna Linea
4400695938 15.09.2025 27.10.2025 Salvini gioielli
Mod. fornitore Ns. modello Descrizione Colore oro U.M. Quantità Importo
VP1991E-WGC2 20085792 collana ob brill. - cuore small ORO BIANCO PZ 2 986,00
Coll. MAGIA
2 USD 986,00
Annotazioni
Per la data di consegna indicata sull'ordine e con Voi concordata, si intende l'accettazione degli oggetti in Casa Damiani, una volta superato il
controllo qualità.
Condizioni di pagamento
Pagina
BONIFICO 120 GG DF
1
Il bonifico bancario verrà effettuato entro 15 giorni dalla scadenza di pagamento, in considerazione della gestione
amm.va e di tesoreria di
Termini di pagamento concordati in deroga, in considerazione della prassi commerciale, tempi di pagamento dei
clienti e natura del prodotto 1
*Il documento di trasporto e la fattura devono necessariamente riportare il numero completo del presente ordine onde evitare impossibilità di
registrazione e pagamento. Si richiede inoltre di mantenere la stessa sequenza riga per riga del presente ordine.
Si dichiara che Damiani s.p.a. svolge la propria attività in conformità alla legge, al Modello Organizzativo e al Codice Etico adottati ai sensi e per gli effetti del D.Lgs. 231/2001 ed esige che proprie
controparti contrattuali si conformino alla predetta normativa e al predetto Codice consultabile sul sito www.damiani.it specificando che l'eventuale comportamento contrario a detti principi potrà
comportare la risoluzione del contratto ai sensi dell'art. 1456 c.c. e l'adozione delle ulteriori misure previste dalla legge.
ORDINE DI ACQUISTO Standard (man)
Destinatario 40501295
Sede: 15048 VALENZA (AL) - P.za Damiano Grassi Damiani, 1 MESS.RS
P.IVA 01457570065 - R.E.A. 162836 AL - 3181 AL PUSHPAM COLLECTION BVBA
R.I.01457570065-Cap.soc. Euro 36.344.000 i.v.-M/AL012786
HOVENIERSSTRAAT 53,SUITE 907-909 9T
Fine Jewelry
B-2018 ANTWERP
- VALENZA 15048
P.za Damiano Grassi Damiani, 1
Tel.(0131) 92.96.11
Fax (0131) 94.16.53
damiani@damiani.it
Persona da contattare/Tel.
- MILANO 20121 VA: UA COLLEZ. SAL
Via Montenapoleone, 10
Federica Carnevale
Tel.(02) 46.71.61
Fax (02) 48.00.83.78 tel. 0131929600
Cd. dest. SDI: 7U7ZY9N
Indirizzo PEC: damiani@damiani.postecert.it
Numero Data Data consegna Linea
4400695939 15.09.2025 27.10.2025 Salvini gioielli
Mod. fornitore Ns. modello Descrizione Colore oro U.M. Quantità Importo
VR1943EEA-WGC2 20086252 anello ob brill. mis.14 - smeraldo ORO BIANCO PZ 1 647,00
Mis. 14 Coll. MAGIA
Annotazioni
Per la data di consegna indicata sull'ordine e con Voi concordata, si intende l'accettazione degli oggetti in Casa Damiani, una volta superato il
controllo qualità.
Condizioni di pagamento
Pagina
BONIFICO 120 GG DF
1
Il bonifico bancario verrà effettuato entro 15 giorni dalla scadenza di pagamento, in considerazione della gestione
amm.va e di tesoreria di
Termini di pagamento concordati in deroga, in considerazione della prassi commerciale, tempi di pagamento dei
clienti e natura del prodotto 1
*Il documento di trasporto e la fattura devono necessariamente riportare il numero completo del presente ordine onde evitare impossibilità di
registrazione e pagamento. Si richiede inoltre di mantenere la stessa sequenza riga per riga del presente ordine.
Si dichiara che Damiani s.p.a. svolge la propria attività in conformità alla legge, al Modello Organizzativo e al Codice Etico adottati ai sensi e per gli effetti del D.Lgs. 231/2001 ed esige che proprie
controparti contrattuali si conformino alla predetta normativa e al predetto Codice consultabile sul sito www.damiani.it specificando che l'eventuale comportamento contrario a detti principi potrà
comportare la risoluzione del contratto ai sensi dell'art. 1456 c.c. e l'adozione delle ulteriori misure previste dalla legge.
ORDINE DI ACQUISTO Standard (man)
Destinatario 40501295
Sede: 15048 VALENZA (AL) - P.za Damiano Grassi Damiani, 1 MESS.RS
P.IVA 01457570065 - R.E.A. 162836 AL - 3181 AL PUSHPAM COLLECTION BVBA
R.I.01457570065-Cap.soc. Euro 36.344.000 i.v.-M/AL012786
HOVENIERSSTRAAT 53,SUITE 907-909 9T
Fine Jewelry
B-2018 ANTWERP
- VALENZA 15048
P.za Damiano Grassi Damiani, 1
Tel.(0131) 92.96.11
Fax (0131) 94.16.53
damiani@damiani.it
Persona da contattare/Tel.
- MILANO 20121 VA: UA COLLEZ. SAL
Via Montenapoleone, 10
Federica Carnevale
Tel.(02) 46.71.61
Fax (02) 48.00.83.78 tel. 0131929600
Cd. dest. SDI: 7U7ZY9N
Indirizzo PEC: damiani@damiani.postecert.it
Numero Data Data consegna Linea
4400695940 15.09.2025 27.10.2025 Salvini gioielli
Mod. fornitore Ns. modello Descrizione Colore oro U.M. Quantità Importo
YR0303C-WG 20094124 anello og brill.mis.13 - smeraldo m ORO GIALLO PZ 1 314,00
Mis. 13 Coll. MAGIA
Annotazioni
Per la data di consegna indicata sull'ordine e con Voi concordata, si intende l'accettazione degli oggetti in Casa Damiani, una volta superato il
controllo qualità.
Condizioni di pagamento
Pagina
BONIFICO 120 GG DF
1
Il bonifico bancario verrà effettuato entro 15 giorni dalla scadenza di pagamento, in considerazione della gestione
amm.va e di tesoreria di
Termini di pagamento concordati in deroga, in considerazione della prassi commerciale, tempi di pagamento dei
clienti e natura del prodotto 1
*Il documento di trasporto e la fattura devono necessariamente riportare il numero completo del presente ordine onde evitare impossibilità di
registrazione e pagamento. Si richiede inoltre di mantenere la stessa sequenza riga per riga del presente ordine.
Si dichiara che Damiani s.p.a. svolge la propria attività in conformità alla legge, al Modello Organizzativo e al Codice Etico adottati ai sensi e per gli effetti del D.Lgs. 231/2001 ed esige che proprie
controparti contrattuali si conformino alla predetta normativa e al predetto Codice consultabile sul sito www.damiani.it specificando che l'eventuale comportamento contrario a detti principi potrà
comportare la risoluzione del contratto ai sensi dell'art. 1456 c.c. e l'adozione delle ulteriori misure previste dalla legge.

In [7]:
#for 915 excel
import pandas as pd
import re

def process_pc2_damiani_data():
    """
    Process the extracted text from PC2 Damiani PDF and create DataFrame with specified columns
    """
    
    # The extracted text from the PDF (from the second cell output)
    extracted_text = """ORDINE DI ACQUISTO Standard (man)
Destinatario 40501295
Sede: 15048 VALENZA (AL) - P.za Damiano Grassi Damiani, 1 MESS.RS
P.IVA 01457570065 - R.E.A. 162836 AL - 3181 AL PUSHPAM COLLECTION BVBA
R.I.01457570065-Cap.soc. Euro 36.344.000 i.v.-M/AL012786
HOVENIERSSTRAAT 53,SUITE 907-909 9T
Fine Jewelry
B-2018 ANTWERP
- VALENZA 15048
P.za Damiano Grassi Damiani, 1
Tel.(0131) 92.96.11
Fax (0131) 94.16.53
damiani@damiani.it
Persona da contattare/Tel.
- MILANO 20121 VA: UA COLLEZ. SAL
Via Montenapoleone, 10
Federica Carnevale
Tel.(02) 46.71.61
Fax (02) 48.00.83.78 tel. 0131929600
Cd. dest. SDI: 7U7ZY9N
Indirizzo PEC: damiani@damiani.postecert.it
Numero Data Data consegna Linea
4400695938 15.09.2025 27.10.2025 Salvini gioielli
Mod. fornitore Ns. modello Descrizione Colore oro U.M. Quantità Importo
VP1991E-WGC2 20085792 collana ob brill. - cuore small ORO BIANCO PZ 2 986,00
Coll. MAGIA
2 USD 986,00
Annotazioni
Per la data di consegna indicata sull'ordine e con Voi concordata, si intende l'accettazione degli oggetti in Casa Damiani, una volta superato il
controllo qualità.
Condizioni di pagamento
Pagina
BONIFICO 120 GG DF
1
Il bonifico bancario verrà effettuato entro 15 giorni dalla scadenza di pagamento, in considerazione della gestione
amm.va e di tesoreria di
Termini di pagamento concordati in deroga, in considerazione della prassi commerciale, tempi di pagamento dei
clienti e natura del prodotto 1
*Il documento di trasporto e la fattura devono necessariamente riportare il numero completo del presente ordine onde evitare impossibilità di
registrazione e pagamento. Si richiede inoltre di mantenere la stessa sequenza riga per riga del presente ordine.
Si dichiara che Damiani s.p.a. svolge la propria attività in conformità alla legge, al Modello Organizzativo e al Codice Etico adottati ai sensi e per gli effetti del D.Lgs. 231/2001 ed esige che proprie
controparti contrattuali si conformino alla predetta normativa e al predetto Codice consultabile sul sito www.damiani.it specificando che l'eventuale comportamento contrario a detti principi potrà
comportare la risoluzione del contratto ai sensi dell'art. 1456 c.c. e l'adozione delle ulteriori misure previste dalla legge.
ORDINE DI ACQUISTO Standard (man)
Destinatario 40501295
Sede: 15048 VALENZA (AL) - P.za Damiano Grassi Damiani, 1 MESS.RS
P.IVA 01457570065 - R.E.A. 162836 AL - 3181 AL PUSHPAM COLLECTION BVBA
R.I.01457570065-Cap.soc. Euro 36.344.000 i.v.-M/AL012786
HOVENIERSSTRAAT 53,SUITE 907-909 9T
Fine Jewelry
B-2018 ANTWERP
- VALENZA 15048
P.za Damiano Grassi Damiani, 1
Tel.(0131) 92.96.11
Fax (0131) 94.16.53
damiani@damiani.it
Persona da contattare/Tel.
- MILANO 20121 VA: UA COLLEZ. SAL
Via Montenapoleone, 10
Federica Carnevale
Tel.(02) 46.71.61
Fax (02) 48.00.83.78 tel. 0131929600
Cd. dest. SDI: 7U7ZY9N
Indirizzo PEC: damiani@damiani.postecert.it
Numero Data Data consegna Linea
4400695939 15.09.2025 27.10.2025 Salvini gioielli
Mod. fornitore Ns. modello Descrizione Colore oro U.M. Quantità Importo
VR1943EEA-WGC2 20086252 anello ob brill. mis.14 - smeraldo ORO BIANCO PZ 1 647,00
Mis. 14 Coll. MAGIA
Annotazioni
Per la data di consegna indicata sull'ordine e con Voi concordata, si intende l'accettazione degli oggetti in Casa Damiani, una volta superato il
controllo qualità.
Condizioni di pagamento
Pagina
BONIFICO 120 GG DF
1
Il bonifico bancario verrà effettuato entro 15 giorni dalla scadenza di pagamento, in considerazione della gestione
amm.va e di tesoreria di
Termini di pagamento concordati in deroga, in considerazione della prassi commerciale, tempi di pagamento dei
clienti e natura del prodotto 1
*Il documento di trasporto e la fattura devono necessariamente riportare il numero completo del presente ordine onde evitare impossibilità di
registrazione e pagamento. Si richiede inoltre di mantenere la stessa sequenza riga per riga del presente ordine.
Si dichiara che Damiani s.p.a. svolge la propria attività in conformità alla legge, al Modello Organizzativo e al Codice Etico adottati ai sensi e per gli effetti del D.Lgs. 231/2001 ed esige che proprie
controparti contrattuali si conformino alla predetta normativa e al predetto Codice consultabile sul sito www.damiani.it specificando che l'eventuale comportamento contrario a detti principi potrà
comportare la risoluzione del contratto ai sensi dell'art. 1456 c.c. e l'adozione delle ulteriori misure previste dalla legge.
ORDINE DI ACQUISTO Standard (man)
Destinatario 40501295
Sede: 15048 VALENZA (AL) - P.za Damiano Grassi Damiani, 1 MESS.RS
P.IVA 01457570065 - R.E.A. 162836 AL - 3181 AL PUSHPAM COLLECTION BVBA
R.I.01457570065-Cap.soc. Euro 36.344.000 i.v.-M/AL012786
HOVENIERSSTRAAT 53,SUITE 907-909 9T
Fine Jewelry
B-2018 ANTWERP
- VALENZA 15048
P.za Damiano Grassi Damiani, 1
Tel.(0131) 92.96.11
Fax (0131) 94.16.53
damiani@damiani.it
Persona da contattare/Tel.
- MILANO 20121 VA: UA COLLEZ. SAL
Via Montenapoleone, 10
Federica Carnevale
Tel.(02) 46.71.61
Fax (02) 48.00.83.78 tel. 0131929600
Cd. dest. SDI: 7U7ZY9N
Indirizzo PEC: damiani@damiani.postecert.it
Numero Data Data consegna Linea
4400695940 15.09.2025 27.10.2025 Salvini gioielli
Mod. fornitore Ns. modello Descrizione Colore oro U.M. Quantità Importo
YR0303C-WG 20094124 anello og brill.mis.13 - smeraldo m ORO GIALLO PZ 1 314,00
Mis. 13 Coll. MAGIA
Annotazioni
Per la data di consegna indicata sull'ordine e con Voi concordata, si intende l'accettazione degli oggetti in Casa Damiani, una volta superato il
controllo qualità.
Condizioni di pagamento
Pagina
BONIFICO 120 GG DF
1
Il bonifico bancario verrà effettuato entro 15 giorni dalla scadenza di pagamento, in considerazione della gestione
amm.va e di tesoreria di
Termini di pagamento concordati in deroga, in considerazione della prassi commerciale, tempi di pagamento dei
clienti e natura del prodotto 1
*Il documento di trasporto e la fattura devono necessariamente riportare il numero completo del presente ordine onde evitare impossibilità di
registrazione e pagamento. Si richiede inoltre di mantenere la stessa sequenza riga per riga del presente ordine.
Si dichiara che Damiani s.p.a. svolge la propria attività in conformità alla legge, al Modello Organizzativo e al Codice Etico adottati ai sensi e per gli effetti del D.Lgs. 231/2001 ed esige che proprie
controparti contrattuali si conformino alla predetta normativa e al predetto Codice consultabile sul sito www.damiani.it specificando che l'eventuale comportamento contrario a detti principi potrà
comportare la risoluzione del contratto ai sensi dell'art. 1456 c.c. e l'adozione delle ulteriori misure previste dalla legge."""
    
    # Initialize the data list
    data = []
    
    # Find the ItemPoNo (Linea number)
    linea_match = re.search(r'(\d+)\s+\d{2}\.\d{2}\.\d{4}\s+\d{2}\.\d{2}\.\d{4}\s+Salvini gioielli', extracted_text)
    item_po_no = linea_match.group(1) if linea_match else ""
    
    # Find the item line after "U.M. Quantità Importo"
    item_line_match = re.search(r'U\.M\. Quantità Importo\n([^\n]+)', extracted_text)
    if item_line_match:
        item_line = item_line_match.group(1).strip()
        
        # Extract StyleCode (first two words)
        words = item_line.split()
        style_code = f"{words[0]} {words[1]}" if len(words) >= 2 else words[0] if words else ""
        
        # Extract ItemRefNo (third word - the number after StyleCode)
        item_ref_no = words[1] if len(words) >= 3 else ""
        
        # Extract ItemSize (number after "mis.")
        size_match = re.search(r'mis\.(\d+)', item_line)
        item_size = f"IT {size_match.group(1)}" if size_match else ""
        
        # Extract Tone (before "PZ")
        tone_match = re.search(r'(ORO\s+\w+)\s+PZ', item_line)
        tone = ""
        if tone_match:
            oro_type = tone_match.group(1)
            if "ORO BIANCO" in oro_type:
                tone = "WG"
            elif "ORO ROSE" in oro_type:
                tone = "PG"
            elif "ORO GIALLO" in oro_type:
                tone = "YG"
        
        # Extract OrderQty (number after "PZ")
        qty_match = re.search(r'PZ\s+(\d+)', item_line)
        order_qty = qty_match.group(1) if qty_match else ""
        
        # Create CustomerProductionInstruction based on tone
        customer_production_instruction = ""
        if tone == "WG":
            customer_production_instruction = "750 WHITE GOLD"
        elif tone == "PG":
            customer_production_instruction = "750 PINK GOLD"
        elif tone == "YG":
            customer_production_instruction = "750 YELLOW GOLD"
        
        # Create DesignProductionInstruction based on tone
        design_production_instruction = ""
        if tone == "WG":
            design_production_instruction = "WHITE RODIUM"
        elif tone == "PG":
            design_production_instruction = "NO RODIUM"
        elif tone == "YG":
            design_production_instruction = "NO RODIUM"
        
        # Create SpecialRemarks
        size_for_remarks = item_size.replace("IT ", "").replace(".", "")  # Remove IT prefix and decimal point
        special_remarks = f"SVP - {item_ref_no}, {customer_production_instruction}, {design_production_instruction}, ITALIAN SIZE - {size_for_remarks}"
        
        # Create the data dictionary with all required columns
        item_data = {
            'SrNo': 1,  # Incremental value starting from 1
            'StyleCode': style_code,
            'ItemSize': item_size,
            'OrderQty': order_qty,
            'OrderItemPcs': "",  # Leave blank
            'Metal': "G750",  # Fixed value
            'Tone': tone,
            'ItemPoNo.': item_po_no,
            'ItemRefNo': item_ref_no,
            'StockType': "",  # Leave blank
            'Priority': "REG",  # Fixed value
            'MakeType': "",  # Leave blank
            'CustomerProductionInstruction': customer_production_instruction,
            'SpecialRemarks': special_remarks,
            'DesignProductionInstruction': design_production_instruction,
            'StampInstruction': "750 SALVINI",  # Fixed value
            'OrderGroup': "",  # Leave blank
            'Certificate': "",  # Leave blank
            'SKUNo': "",  # Leave blank
            'Basestoneminwt': "",  # Leave blank
            'Basestonemaxwt': "",  # Leave blank
            'Basemetalminwt': "",  # Leave blank
            'Basemetalmaxwt': "",  # Leave blank
            'Productiondeliverydate': "",  # Leave blank
            'Expecteddeliverydate': "",  # Leave blank
            'SetPrice': "",  # Leave blank
            'StoneQuality': ""  # Leave blank
        }
        
        data.append(item_data)
        
        # Debug output to verify extraction
        print("Debug - Extracted values:")
        print(f"ItemPoNo: {item_po_no}")
        print(f"StyleCode: {style_code}")
        print(f"ItemRefNo: {item_ref_no}")
        print(f"ItemSize: {item_size}")
        print(f"Tone: {tone}")
        print(f"OrderQty: {order_qty}")
        print(f"CustomerProductionInstruction: {customer_production_instruction}")
        print(f"DesignProductionInstruction: {design_production_instruction}")
        print(f"SpecialRemarks: {special_remarks}")
        print("-" * 50)
    
    return data

# Process the data
processed_data = process_pc2_damiani_data()

# Create DataFrame
df = pd.DataFrame(processed_data)

# Display the DataFrame
print("Processed DataFrame:")
print("=" * 80)
print(df.to_string(index=False))
print("=" * 80)

# Save to Excel file
output_filename = 'PC2_250915_Cleaned.xlsx'
df.to_excel(output_filename, index=False)

print(f"\nData successfully saved to '{output_filename}'")
print(f"Total items processed: {len(df)}")
print(f"Columns created: {len(df.columns)}")

# Display column names
print(f"\nColumn names:")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

df


Debug - Extracted values:
ItemPoNo: 4400695938
StyleCode: VP1991E-WGC2 20085792
ItemRefNo: 20085792
ItemSize: 
Tone: WG
OrderQty: 2
CustomerProductionInstruction: 750 WHITE GOLD
DesignProductionInstruction: WHITE RODIUM
SpecialRemarks: SVP - 20085792, 750 WHITE GOLD, WHITE RODIUM, ITALIAN SIZE - 
--------------------------------------------------
Processed DataFrame:
 SrNo             StyleCode ItemSize OrderQty OrderItemPcs Metal Tone  ItemPoNo. ItemRefNo StockType Priority MakeType CustomerProductionInstruction                                                SpecialRemarks DesignProductionInstruction StampInstruction OrderGroup Certificate SKUNo Basestoneminwt Basestonemaxwt Basemetalminwt Basemetalmaxwt Productiondeliverydate Expecteddeliverydate SetPrice StoneQuality
    1 VP1991E-WGC2 20085792                 2               G750   WG 4400695938  20085792                REG                         750 WHITE GOLD SVP - 20085792, 750 WHITE GOLD, WHITE RODIUM, ITALIAN SIZE -          

,SrNo,StyleCode,ItemSize,OrderQty,OrderItemPcs,Metal,Tone,ItemPoNo.,ItemRefNo,StockType,...,Certificate,SKUNo,Basestoneminwt,Basestonemaxwt,Basemetalminwt,Basemetalmaxwt,Productiondeliverydate,Expecteddeliverydate,SetPrice,StoneQuality
0,1,VP1991E-WGC2 20085792,,2,,G750,WG,4400695938,20085792,,...,,,,,,,,,,


In [ ]:
#FINAL CODE
# Enhanced Data processing for PC2 - Damiani Purchase Order (Multiple Items)
import pandas as pd
import re

def process_pc2_damiani_multiple_items():
    """
    Process the extracted text from PC2 Damiani PDF and create DataFrame with multiple items.
    Each item section ends when 'ulteriori misure previste dalla legge.' is encountered.
    """
    
    # --- Extracted text from PDF (example for testing) ---
    extracted_text = """250915
ORDINE DI ACQUISTO Standard (man)
Destinatario 40501295
Sede: 15048 VALENZA (AL) - P.za Damiano Grassi Damiani, 1 MESS.RS
P.IVA 01457570065 - R.E.A. 162836 AL - 3181 AL PUSHPAM COLLECTION BVBA
R.I.01457570065-Cap.soc. Euro 36.344.000 i.v.-M/AL012786
HOVENIERSSTRAAT 53,SUITE 907-909 9T
Fine Jewelry
B-2018 ANTWERP
- VALENZA 15048
P.za Damiano Grassi Damiani, 1
Tel.(0131) 92.96.11
Fax (0131) 94.16.53
damiani@damiani.it
Persona da contattare/Tel.
- MILANO 20121 VA: UA COLLEZ. SAL
Via Montenapoleone, 10
Federica Carnevale
Tel.(02) 46.71.61
Fax (02) 48.00.83.78 tel. 0131929600
Cd. dest. SDI: 7U7ZY9N
Indirizzo PEC: damiani@damiani.postecert.it
Numero Data Data consegna Linea
4400695938 15.09.2025 27.10.2025 Salvini gioielli
Mod. fornitore Ns. modello Descrizione Colore oro U.M. Quantità Importo
VP1991E-WGC2 20085792 collana ob brill. - cuore small ORO BIANCO PZ 2 986,00
Coll. MAGIA
2 USD 986,00
...
ulteriori misure previste dalla legge.
ORDINE DI ACQUISTO Standard (man)
4400695939 15.09.2025 27.10.2025 Salvini gioielli
Mod. fornitore Ns. modello Descrizione Colore oro U.M. Quantità Importo
VR1943EEA-WGC2 20086252 anello ob brill. mis.14 - smeraldo ORO BIANCO PZ 1 647,00
Mis. 14 Coll. MAGIA
ulteriori misure previste dalla legge.
ORDINE DI ACQUISTO Standard (man)
4400695940 15.09.2025 27.10.2025 Salvini gioielli
Mod. fornitore Ns. modello Descrizione Colore oro U.M. Quantità Importo
YR0303C-WG 20094124 anello og brill.mis.13 - smeraldo m ORO GIALLO PZ 1 314,00
Mis. 13 Coll. MAGIA
ulteriori misure previste dalla legge.
"""
    
    data = []
    sr_no = 1

    # --- Find all item sections ---
    item_sections = re.findall(
        r'U\.M\. Quantità Importo\n(.*?)ulteriori misure previste dalla legge\.',
        extracted_text,
        re.DOTALL
    )

    print(f"DEBUG: Found {len(item_sections)} item section(s)\n")

    # --- Loop through each section ---
    for section in item_sections:
        lines = section.split('\n')
        current_item_lines = []

        for line in lines:
            line = line.strip()
            if not line:
                continue

            # Detect line with item info
            item_pattern = r'^[A-Z0-9\-]+\s+\d+\s+.*?ORO\s+\w+\s+PZ\s+\d+'
            if re.match(item_pattern, line):
                # Save previous item if exists
                if current_item_lines:
                    item_data = process_single_item(current_item_lines, sr_no, "")
                    if item_data:
                        data.append(item_data)
                        sr_no += 1
                current_item_lines = [line]
            elif current_item_lines:
                current_item_lines.append(line)

        # Process last item in this section
        if current_item_lines:
            item_data = process_single_item(current_item_lines, sr_no, "")
            if item_data:
                data.append(item_data)
                sr_no += 1

    return data


def process_single_item(item_lines, sr_no, item_po_no):
    """
    Extract structured fields from a single item block
    """
    item_text = ' '.join(item_lines)
    words = item_text.split()

    if len(words) < 2:
        return None

    style_code = words[0]
    item_ref_no = words[1]

    # Extract Italian ring size
    size_match = re.search(r'mis\.(\d+)', item_text)
    item_size = f"IT {size_match.group(1)}" if size_match else ""

    # Tone extraction
    tone_match = re.search(r'(ORO\s+\w+)\s+PZ', item_text)
    tone = ""
    if tone_match:
        oro_type = tone_match.group(1)
        if "ORO BIANCO" in oro_type:
            tone = "WG"
        elif "ORO ROSE" in oro_type:
            tone = "PG"
        elif "ORO GIALLO" in oro_type:
            tone = "YG"

    # Quantity
    qty_match = re.search(r'PZ\s+(\d+)', item_text)
    order_qty = qty_match.group(1) if qty_match else ""

    # Production instructions
    customer_production_instruction = {
        "WG": "750 WHITE GOLD",
        "PG": "750 PINK GOLD",
        "YG": "750 YELLOW GOLD"
    }.get(tone, "")

    design_production_instruction = {
        "WG": "WHITE RODIUM",
        "PG": "NO RODIUM",
        "YG": "NO RODIUM"
    }.get(tone, "")

    size_for_remarks = item_size.replace("IT ", "").replace(".", "")
    special_remarks = f"SVP - {item_ref_no}, {customer_production_instruction}, {design_production_instruction}, ITALIAN SIZE - {size_for_remarks}"

    return {
        'SrNo': sr_no,
        'StyleCode': style_code,
        'ItemSize': item_size,
        'OrderQty': order_qty,
        'OrderItemPcs': "",
        'Metal': "G750",
        'Tone': tone,
        'ItemPoNo.': item_po_no,
        'ItemRefNo': item_ref_no,
        'StockType': "",
        'Priority': "REG",
        'MakeType': "",
        'CustomerProductionInstruction': customer_production_instruction,
        'SpecialRemarks': special_remarks,
        'DesignProductionInstruction': design_production_instruction,
        'StampInstruction': "750 SALVINI",
        'OrderGroup': "",
        'Certificate': "",
        'SKUNo': "",
        'Basestoneminwt': "",
        'Basestonemaxwt': "",
        'Basemetalminwt': "",
        'Basemetalmaxwt': "",
        'Productiondeliverydate': "",
        'Expecteddeliverydate': "",
        'SetPrice': "",
        'StoneQuality': ""
    }


# --- Main execution ---
processed_data = process_pc2_damiani_multiple_items()
df = pd.DataFrame(processed_data)

print("\nProcessed DataFrame (Multiple Items):")
print("=" * 80)
print(df.to_string(index=False))
print("=" * 80)

output_filename = 'PC2_Damiani_Multiple_Items_Processed_Data.xlsx'
df.to_excel(output_filename, index=False)

print(f"\n✅ Data successfully saved to '{output_filename}'")
print(f"✅ Total items processed: {len(df)}")
print(f"✅ Columns created: {len(df.columns)}")

print("\nColumn names:")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

print("\nDebug - Items found:")
for i, item in enumerate(processed_data, 1):
    print(f"Item {i}:")
    print(f"  SrNo: {item['SrNo']}")
    print(f"  StyleCode: {item['StyleCode']}")
    print(f"  ItemRefNo: {item['ItemRefNo']}")
    print(f"  ItemSize: {item['ItemSize']}")
    print(f"  Tone: {item['Tone']}")
    print(f"  OrderQty: {item['OrderQty']}")
    print(f"  SpecialRemarks: {item['SpecialRemarks']}")
    print("-" * 40)

df


DEBUG: Found 3 item section(s)


Processed DataFrame (Multiple Items):
 SrNo      StyleCode ItemSize OrderQty OrderItemPcs Metal Tone ItemPoNo. ItemRefNo StockType Priority MakeType CustomerProductionInstruction                                                  SpecialRemarks DesignProductionInstruction StampInstruction OrderGroup Certificate SKUNo Basestoneminwt Basestonemaxwt Basemetalminwt Basemetalmaxwt Productiondeliverydate Expecteddeliverydate SetPrice StoneQuality
    1   VP1991E-WGC2                 2               G750   WG            20085792                REG                         750 WHITE GOLD   SVP - 20085792, 750 WHITE GOLD, WHITE RODIUM, ITALIAN SIZE -                 WHITE RODIUM      750 SALVINI                                                                                                                                                           
    2 VR1943EEA-WGC2    IT 14        1               G750   WG            20086252                REG                  

,SrNo,StyleCode,ItemSize,OrderQty,OrderItemPcs,Metal,Tone,ItemPoNo.,ItemRefNo,StockType,...,Certificate,SKUNo,Basestoneminwt,Basestonemaxwt,Basemetalminwt,Basemetalmaxwt,Productiondeliverydate,Expecteddeliverydate,SetPrice,StoneQuality
0,1,VP1991E-WGC2,,2,,G750,WG,,20085792,,...,,,,,,,,,,
1,2,VR1943EEA-WGC2,IT 14,1,,G750,WG,,20086252,,...,,,,,,,,,,
2,3,YR0303C-WG,IT 13,1,,G750,YG,,20094124,,...,,,,,,,,,,
